In [4]:
import numpy as np

from keras.models import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from models.unet import *
from datahandler import DataHandler

import os
import cv2
import skimage.io as io
from tqdm import tqdm
from math import ceil
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_score
from models.unet.unet import *
#from tta_wrapper import tta_segmentation

import warnings
warnings.filterwarnings("ignore")

plt.gray()

<Figure size 640x480 with 0 Axes>

In [5]:
model = getUnet()
model.load_weights('logs/unet/kfold_unet/gamma/gamma_weights.h5')

dh = DataHandler()
images, masks = dh.getData(only_test = True)

2022-12-23 00:54:01.408339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 00:54:14.759475: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4660 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-12-23 00:54:14.765104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 2705 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6
2022-12-23 00:54:14.805784: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/tas

In [6]:
def resetSeed():
    np.random.seed(1)

def getGenerator(images, bs=32):
    resetSeed()

    image_datagen = ImageDataGenerator(rescale=1./255)
    image_datagen.fit(images, augment = True)
    image_generator = image_datagen.flow(x = images, batch_size=bs,
            shuffle = False)

    return image_generator

In [12]:
def dice_coef(y_true, y_pred):     
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    intersection = np.sum(y_pred[y_true==1])
    insum = np.sum(y_true) + np.sum(y_pred)
    
    if insum == 0:
        return 1
    
    return 2. * intersection / insum

def evaluateModel(masks, predictions):
    dice_scores = []
    iou_scores = []
    f1_scores = []
    
    for i, pred in enumerate(tqdm(results, desc='evaluating')):
        mask = masks[i]
        
        mask[mask==255] = 1
        pred[pred<0.5] = 0
        pred[pred>=0.5] = 1

        mask = np.asarray(mask, dtype=np.bool).flatten()
        pred = np.asarray(pred, dtype=np.bool).flatten()
        
        dice = dice_coef(mask,pred)
        iou = jaccard_score(mask, pred)
        f1 = f1_score(mask, pred)
        
        if dice > 0:
            dice_scores.append(dice_coef(mask, pred))
        if iou > 0:
            iou_scores.append(jaccard_score(mask, pred))
        if f1 > 0:
            f1_scores.append(f1_score(mask, pred))
    
    print('dice: %f'%np.mean(dice_scores))
    print('F1: %f'%np.mean(f1_scores))    
    print('IoU: %f'%np.mean(iou_scores))

In [8]:
test_gen = getGenerator(images)
results = model.predict_generator(test_gen, ceil(len(images) / 32), verbose = 1)

2022-12-23 01:00:15.779480: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2022-12-23 01:00:16.659369: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  1/174 [..............................] - ETA: 26:01

2022-12-23 01:00:23.072179: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


174/174 [==============================] - 34s 146ms/step


### Normal

In [13]:
evaluateModel(masks, results)

evaluating: 100%|██████████| 5542/5542 [03:04<00:00, 29.96it/s]

dice: 0.915961
F1: 0.886854
IoU: 0.815101


### Comp vision techniques

In [7]:
def chooseComponent(image, j):
    image = image.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)
    sizes = stats[:, -1]
 
    max_label = 1
    if len(sizes) < 3:
        return image
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
 
    new_img = np.zeros(output.shape)
    new_img[output == max_label] = 1
    return new_img

results_one_comp = []
for i, res in enumerate(tqdm(results, desc='Removing components')):
    results_one_comp.append(chooseComponent(res, i))
evaluateModel(masks, results_one_comp)  

evaluating: 100%|██████████| 136/136 [00:01<00:00, 101.88it/s]

dice: 0.781105
F1: 0.616933
IoU: 0.984857


### TTA

In [8]:

params = dict(
        h_flip=True,
        v_flip=True,
        h_shift=(10, -10),
        v_shift=(10, -10),
        rotation=(90, 180, 270),
        merge='mean')

tta_model = tta_segmentation(model, **params)


In [9]:
results = []

test_gen = getGenerator(images, bs=1)
results = tta_model.predict_generator(test_gen, len(images), verbose = 1)

136/136 [==============================] - 93s 686ms/step


In [10]:
evaluateModel(masks, results)

evaluating: 100%|██████████| 136/136 [00:01<00:00, 96.42it/s] 

dice: 0.820930
F1: 0.648372
IoU: 0.987869
